# Order Status

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import DateEntry
from datetime import datetime
import csv

# Client list with "All"
client_names = ["All", "John Doe", "Jane Smith", "XYZ Corp", "XWT Corp", "XQR Corp", "XGF Corp", "XIU Corp", "XTU Corp", "XOP Corp", "XRT Corp", "PUT Corp"]

# Dummy Data: (Bill No, Client ID, Client Name, Customer Type, Status, Amount, Time)
dummy_data = [
    (101, "CL001", "John Doe", "Retail", "Fabrication", "$1500", "08/02/25 10:30"),
    (102, "CL002", "Jane Smith", "Wholesale", "Testing", "$3200", "08/02/25 11:00"),
    (103, "CL003", "XYZ Corp", "Corporate", "Delivered", "$7800", "08/02/25 12:16"),
    (104, "CL004", "XWT Corp", "Corporate", "Delivered", "$7800", "08/02/25 12:17"),
    (105, "CL005", "XQR Corp", "Corporate", "Delivered", "$7800", "08/02/25 12:18"),
    (106, "CL006", "XGF Corp", "Corporate", "Delivered", "$7800", "08/02/25 01:00"),
    (107, "CL007", "XIU Corp", "Corporate", "Delivered", "$7800", "10/02/25 12:15"),
    (108, "CL008", "XTU Corp", "Corporate", "Delivered", "$7800", "11/02/25 12:15"),
    (109, "CL009", "XOP Corp", "Corporate", "Delivered", "$7890", "12/02/25 12:15"),
    (110, "CL010", "XRT Corp", "Corporate", "Delivered", "$7876", "13/02/25 12:15"),
    (111, "CL011", "PUT Corp", "Corporate", "Delivered", "$7812", "14/02/25 12:15"),
]

# Flow stages in order
flow_stages = ["Order", "Fabrication", "Powder Coating", "Assembly", "Testing", "Ready for Dispatch", "Shipped", "Delivered"]
stage_index_map = {stage: idx for idx, stage in enumerate(flow_stages)}

# Create main window
root = tk.Tk()
root.title("Order Status Tracker")
root.geometry("1200x750")
root.configure(bg='#f0f4f7')

# Header Frame
header_frame = tk.Frame(root, bg='#d1e0e0', pady=10)
header_frame.pack(fill='x')

tk.Label(header_frame, text="Client Name:", bg='#d1e0e0', font=('Arial', 13, 'bold')).pack(side='left', padx=(10, 2))
client_entry = ttk.Combobox(header_frame, width=20)
client_entry['values'] = client_names
client_entry.set("All")
client_entry.pack(side='left')

from_date = DateEntry(header_frame, width=12, date_pattern='dd/mm/yy')
to_date = DateEntry(header_frame, width=12, date_pattern='dd/mm/yy')

# Treeview Frame
table_frame = tk.Frame(root)
table_frame.pack(fill='both', expand=True, pady=10)

columns = ("Select", "Bill No", "Client ID", "Client Name", "Customer Type", "Status", "Amount", "Time")
tree = ttk.Treeview(table_frame, columns=columns, show='headings')

for col in columns:
    tree.heading(col, text=col, anchor='center')
    tree.column(col, anchor='center', width=120)

def load_data(filtered_data):
    for item in tree.get_children():
        tree.delete(item)
    for row in filtered_data:
        values = ("",) + row
        tree.insert("", tk.END, values=values)

load_data(dummy_data)

def filter_by_client():
    name = client_entry.get()
    if not name:
        messagebox.showwarning("Input Required", "Please select a client name.")
        return
    if name == "All":
        load_data(dummy_data)
    else:
        filtered = [row for row in dummy_data if row[2] == name]
        load_data(filtered)

def filter_by_date():
    from_dt = datetime.strptime(from_date.get(), "%d/%m/%y")
    to_dt = datetime.strptime(to_date.get(), "%d/%m/%y")
    selected_client = client_entry.get()

    filtered = []

    for row in dummy_data:
        order_time = datetime.strptime(row[6], "%d/%m/%y %H:%M")
        status = row[4]

        if status == "Delivered" and from_dt <= order_time <= to_dt:
            if selected_client == "All" or row[2] == selected_client:
                filtered.append(row)

    if not filtered:
        messagebox.showinfo("No Results", "No delivered orders found for the selected filters.")

    load_data(filtered)

def view_all():
    load_data(dummy_data)

def view_delivered():
    delivered = [row for row in dummy_data if row[4] == "Delivered"]
    load_data(delivered)

def export_data():
    file_path = tk.filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
    if not file_path:
        return
    try:
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(columns[1:])
            for child in tree.get_children():
                row = tree.item(child)['values'][1:]
                writer.writerow(row)
        messagebox.showinfo("Success", "Data exported successfully!")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to export data.\n{e}")

tk.Button(header_frame, text="Search", bg='#007acc', fg='white', command=filter_by_client).pack(side='left', padx=10)
tk.Label(header_frame, text="From:", bg='#d1e0e0', font=('Arial', 13, 'bold')).pack(side='left')
from_date.pack(side='left', padx=5)
tk.Label(header_frame, text="To:", bg='#d1e0e0', font=('Arial', 13, 'bold')).pack(side='left')
to_date.pack(side='left', padx=5)
tk.Button(header_frame, text="Filter", bg='#007acc', fg='white', command=filter_by_date).pack(side='left', padx=10)
tk.Button(header_frame, text="Show Details", bg='green', fg='white').pack(side='left', padx=10)

tree.pack(fill='both', expand=True)

# Status Buttons Frame
status_frame = tk.LabelFrame(root, text="Update Status", bg='#f0f4f7', font=('Arial', 13, 'bold'))
status_frame.pack(fill='x', padx=10, pady=5)

# Flow tracker frame (bottom)
flow_frame = tk.Frame(root, bg='#ffffff', pady=10)
flow_frame.pack(fill='x', padx=10, pady=10)

flow_stage_labels = {}

def create_flow_tracker():
    for widget in flow_frame.winfo_children():
        widget.destroy()
    for stage in flow_stages:
        lbl = tk.Label(flow_frame, text=stage, bg='lightgray', font=('Arial', 13, 'bold'), width=16, height=2, relief='ridge', bd=2)
        lbl.pack(side='left', padx=(5, 0))
        flow_stage_labels[stage] = lbl
        if stage != flow_stages[-1]:
            arrow = tk.Label(flow_frame, text='➔', font=('Arial', 13, 'bold'), bg='#ffffff')
            arrow.pack(side='left', padx=(0, 5))

create_flow_tracker()

# To track which stages have been clicked per order (key: Treeview item ID, value: set of stages clicked)
clicked_stages = {}

def update_flow_tracker(current_stage):
    # Highlight stages up to current_stage green
    current_index = stage_index_map.get(current_stage, -1)
    for idx, stage in enumerate(flow_stages):
        if idx < current_index:
            flow_stage_labels[stage].config(bg='green', fg='white')  # Completed stages green
        elif idx == current_index:
            flow_stage_labels[stage].config(bg='skyblue', fg='white')  # Current stage blue
        else:
            flow_stage_labels[stage].config(bg='lightgray', fg='black')  # Pending stages gray

def show_stage_details(stage, order_data):
    # Show all stages from "Order" up to given stage for this order/client
    current_idx = stage_index_map.get(stage, -1)
    stages_up_to = flow_stages[:current_idx + 1]
    details = f"Details for client: {order_data[2]} (Bill No: {order_data[0]})\n\n"
    details += "Completed Stages:\n"
    for s in stages_up_to:
        details += f"- {s}\n"
    messagebox.showinfo(f"Stage '{stage}' Details", details)

def on_stage_button_click(stage):
    selected_items = tree.selection()
    if not selected_items:
        messagebox.showwarning("Warning", "Please select an order from the table.")
        return
    item = selected_items[0]
    order = tree.item(item)['values'][1:]  # exclude "Select" column
    current_status = order[4]

    # Initialize clicked stages for this item if not exists
    if item not in clicked_stages:
        clicked_stages[item] = set()

    # If this stage was already clicked for this order, ignore click
    if stage in clicked_stages[item]:
        messagebox.showinfo("Info", f"Stage '{stage}' was already processed for this order.")
        return

    # If the clicked stage index is less than or equal to current order status index, allow updating and showing details
    order_status_index = stage_index_map.get(current_status, -1)
    clicked_stage_index = stage_index_map.get(stage, -1)

    if clicked_stage_index <= order_status_index:
        # Show details up to clicked stage
        show_stage_details(stage, order)
        # Mark stage as clicked (processed)
        clicked_stages[item].add(stage)
        # Update Treeview status to this stage if stage is ahead of current
        if clicked_stage_index > order_status_index:
            new_values = list(order)
            new_values[4] = stage  # update status
            tree.item(item, values=("", *new_values))
        # Update flow tracker for this stage
        update_flow_tracker(stage)
    else:
        messagebox.showwarning("Invalid", f"You cannot process stage '{stage}' before completing previous stages.")

# Create stage buttons
for stage in flow_stages:
    if stage == "Order":  # skip order button for status updates
        continue
    btn = tk.Button(status_frame, text=stage, bg='#007acc', fg='white', width=15, font=('Arial', 13, 'bold'),
                    command=lambda s=stage: on_stage_button_click(s))
    btn.pack(side='left', padx=5, pady=5)

# Bottom buttons frame
bottom_frame = tk.Frame(root, bg='#f0f4f7')
bottom_frame.pack(fill='x', pady=5)


tk.Button(bottom_frame, text="Export Data", bg='purple', fg='white', width=15, font=('Arial', 13, 'bold'), command=export_data).pack(side='right', padx=10)
tk.Button(bottom_frame, text="View Delivered", bg='green', fg='white', width=15, font=('Arial', 13, 'bold'), command=view_delivered).pack(side='right', padx=10)
tk.Button(bottom_frame, text="View All", bg='blue', fg='white', width=15, font=('Arial', 13, 'bold'), command=view_all).pack(side='right', padx=10)

root.mainloop() 
